In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import itertools
import sklearn.feature_selection
import sklearn.model_selection as ms
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy.stats import skew
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV
from scipy import stats
from sklearn.preprocessing import normalize
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
plt.style.use('ggplot')
pd.options.display.max_columns = None
pd.options.display.max_rows = 600

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Adding 'type' columns that contains 'train' or 'test', and add empty SalePrice columns to the test part
train.insert(0, 'type', 'train')
test.insert(0,'type','test')
test['SalePrice'] = 0.0

# merging the two dataframes
merged = pd.concat([train,test],axis=0).reset_index(drop=True)
merged['SalePrice'] = merged['SalePrice'].astype('float')

# Data cleaning
import Dprocessv2
train = Dprocessv2.dataProcess(merged);

In [4]:
# numerical and categorical features + EXCLUDED FIRST
excl_features = merged[['Id','type','SalePrice']].columns
cat_features = merged.select_dtypes(include = ["object"]).columns.drop('type')
num_features = merged.select_dtypes(exclude = ["object"]).columns.drop(['Id','SalePrice'])

mergedNum = merged[num_features]
mergedCat = merged[cat_features]
mergedExc = merged[excl_features]

mergedNAtotal = merged.isnull().sum().sort_values(ascending=False)

print("NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

if mergedNum.isnull().values.sum() != 0:
    mergedNum = mergedNum.fillna(mergedNum.median())
    print("Remaining NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

NAs for numerical features in merged : 0


In [5]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in merged.columns:
    if merged[i].dtype in numeric_dtypes: 
        numerics2.append(i)

skew_features = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews = pd.DataFrame({'skew':skew_features})

In [6]:
high_skew = skew_features[skew_features > 0.5]
high_skew = high_skew
skew_index = high_skew.index

for i in skew_index:
    if i != 'SalePrice':
        merged[i]= boxcox1p(merged[i], boxcox_normmax(merged[i]+1))

        
skew_features2 = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews2 = pd.DataFrame({'skew':skew_features2})

/Users/macpro/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [7]:
# Create dummy features for categorical values via one-hot encoding

print("Remaining NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

print("NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

mergedCat = pd.get_dummies(mergedCat)


Remaining NAs for categorical features in merged : 0
NAs for categorical features in merged : 0


In [8]:
# rejoin excluded, categorical and numerical features
merged = pd.concat([mergedExc,mergedNum, mergedCat], axis = 1)

print("New number of features : " + str(merged.shape[1]))

New number of features : 292


In [9]:
merged.shape

(2916, 292)

In [10]:
train = merged[merged['type'] == 'train']
train.drop(columns=['Id','type'],inplace=True)
test = merged[merged['type'] == 'test']
test.drop(columns=['type','SalePrice'],inplace=True)

/Users/macpro/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
# taking the log of the saleprice

train['SalePrice'] = np.log(train['SalePrice'])
train['SalePrice'].describe()

/Users/macpro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count    1457.000000
mean       12.023714
std         0.399695
min        10.460242
25%        11.774520
50%        12.001505
75%        12.273731
max        13.534473
Name: SalePrice, dtype: float64

### Linear Regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import sklearn.model_selection as ms
from scipy.stats import norm
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import (GradientBoostingRegressor , RandomForestRegressor,
                              BaggingRegressor,ExtraTreesRegressor,AdaBoostRegressor)
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [ ]:
y = train.SalePrice

## Defining the train/test split (80/20)

In [12]:
house_features = train.drop('SalePrice',axis = 1)
prices = train['SalePrice']

x_train, x_test, y_train, y_test = ms.train_test_split(house_features, prices, 
                                                       test_size=1.0/5, random_state=111)

## Ridge

In [13]:
def goridcv(x, y, alphas_=np.linspace(1e-5,20,10000),
            normalize_=True,
            scoring_=None,
            verbose_=True):
    
    global ridcv
    ridcv = RidgeCV(alphas=alphas_, normalize=normalize_, scoring=scoring_)
    ridcv.fit(x, y)
    
    if verbose_ == True:
        print('The best alpha is: {:.6f}'.format(ridcv.alpha_))
        print('\n')
        print('The RMSE is: {}'.format(mean_squared_error(y_true=y, y_pred=ridcv.predict(x))))
        print('\n')
        print('The ridge intercept is: {:.2f}'.format(ridcv.intercept_))
        print('\n')
        print(pd.Series(ridcv.coef_, index=x.columns))
        print('\n')
        print('The model score is {:.4f}'.format(ridcv.score(x, y)))
        
    return ridcv

In [14]:
goridcv(x_train,y_train)

The best alpha is: 0.230033


The RMSE is: 0.009096510285161062


The ridge intercept is: 7.83


LotFrontage              0.000607
LotArea                  0.000002
OverallQual              0.030261
OverallCond              0.022356
YearBuilt                0.000449
YearRemodAdd             0.000657
MasVnrArea               0.000044
ExterQual                0.021452
ExterCond               -0.003655
BsmtQual                 0.013824
BsmtCond                 0.008524
BsmtFinSF1               0.000055
BsmtFinSF2               0.000029
BsmtUnfSF                0.000008
TotalBsmtSF              0.000071
HeatingQC                0.011261
1stFlrSF                 0.000089
2ndFlrSF                 0.000049
LowQualFinSF            -0.000002
GrLivArea                0.000086
BsmtFullBath             0.021169
BsmtHalfBath             0.004105
FullBath                 0.035199
HalfBath                 0.031139
BedroomAbvGr             0.003516
KitchenAbvGr            -0.018924
KitchenQual        

RidgeCV(alphas=array([1.0000e-05, 2.0102e-03, ..., 1.9998e+01, 2.0000e+01]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=True,
    scoring=None, store_cv_values=False)

In [15]:
mean_squared_error(y_true=y_test, y_pred=ridcv.predict(x_test))

0.013035277778544725

In [16]:
pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}).to_csv('outv2.csv',index=False)